# Práctica Deep Learning - KeepCoding

## Predicción del precio de la vivienda en Airbnb

### 1. Importación de los datos


In [1]:
import cv2
import numpy
import pandas
import imageio.v3 as io

from tqdm import tqdm
from typing import Optional, Union
data = pandas.read_csv("data/dataset.csv", sep = ';')
data = data.dropna(subset=['Price'])
data.head()

ModuleNotFoundError: No module named 'cv2'